# Data Prep & Sync

In [ ]:
import os

def remove_string_from_filenames(directory, string_to_remove):
    """
    Removes a specific string from the beginning of .jpg file names in the given directory.
    
    Parameters:
    - directory (str): Path to the directory containing the files.
    - string_to_remove (str): The string to be removed from the start of file names.
    """
    for filename in os.listdir(directory):
        # Check if the file has a .jpg extension
        if filename.endswith(".jpg") and filename.startswith(string_to_remove):
            # Remove the specified string from the start of the filename
            new_name = filename[len(string_to_remove):]
            # Construct full file paths
            old_path = os.path.join(directory, filename)
            new_path = os.path.join(directory, new_name)
            # Rename the file
            os.rename(old_path, new_path)
            print(f"Renamed: {filename} -> {new_name}")
"""
# Specify the directory and string to remove
directory_path = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\labeled_regions\inside"  # Replace with your directory path
string_to_remove = "mask_inside_"      # Replace with the string to remove

# Call the function
remove_string_from_filenames(directory_path, string_to_remove)
"""

In [ ]:
import os
import pandas as pd

# File paths
opc_data_path = r"D:\PAIND\DATA\20240527_Data_Prozess_01\opcua\opc-ua_data.csv"
webcam_folder = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\labeled_regions\inside_downscaled"
zed2_folder = r"D:\PAIND\DATA\20240527_Data_Prozess_01\zed2"

# Load and preprocess the OPC data
def load_opc_data(opc_path, custom_headers= [
    "Timestamp", "Parameter1", "Parameter2", "Parameter3", "Parameter4", 
    "Parameter5", "Parameter6", "Parameter7", "Parameter8", "Parameter9", 
    "Flag1", "Flag2", "Flag3", "Flag4", "Flag5", "Flag6", "Parameter10", 
    "Flag7", "Flag8", "Flag9", "Flag10", "Flag11", "Parameter11"
]):
    try:
        # Read the file, specifying ';' as the delimiter and without headers
        opc_df = pd.read_csv(opc_path, sep=';', header=None)

        # Assign custom headers to the DataFrame
        opc_df.columns = custom_headers

        print(f"Preprocessed OPC data with {len(custom_headers)} custom headers.")
        return opc_df
    except Exception as e:
        print(f"Error preprocessing OPC data: {e}")
        return None
# Load image file names into a DataFrame
def load_image_data(image_folder, camera_name):
    try:
        image_files = sorted([f for f in os.listdir(image_folder) if f.endswith(".jpg")])
        camera_df = pd.DataFrame(image_files, columns=["image_name"])
        camera_df["camera"] = camera_name
        print(f"Loaded {len(camera_df)} images for {camera_name}.")
        return camera_df
    except Exception as e:
        print(f"Error loading images for {camera_name}: {e}")
        return None

# Load OPC data
opc_df = load_opc_data(opc_data_path)

# Load webcam images into a DataFrame
webcam_df = load_image_data(webcam_folder, "webcam")

# Load Zed2 images into a DataFrame
zed2_df = load_image_data(zed2_folder, "zed2")

# Display summary of loaded data
if opc_df is not None:
    print("OPC DataFrame Head:")
    print(opc_df)

if webcam_df is not None:
    print("Webcam DataFrame Head:")
    print(webcam_df)

if zed2_df is not None:
    print("Zed2 DataFrame Head:")
    print(zed2_df)


In [ ]:
# Extract and process timestamps from webcam images
webcam_df["Timestamp"] = webcam_df["image_name"].str.extract(r"(\d{8}_\d{9})")
webcam_df["Timestamp"] = pd.to_datetime(webcam_df["Timestamp"], format="%Y%m%d_%H%M%S%f")

# Extract and process timestamps from ZED2 images
zed2_df["Timestamp"] = zed2_df["image_name"].str.extract(r"(\d{8}_\d{9})")
zed2_df["Timestamp"] = pd.to_datetime(zed2_df["Timestamp"], format="%Y%m%d_%H%M%S%f")

# Combine the webcam and ZED2 data into a single DataFrame
images_df = pd.concat([webcam_df, zed2_df], ignore_index=True)

# Process OPC DataFrame timestamps
opc_df["Timestamp"] = pd.to_datetime(opc_df["Timestamp"], format="%Y%m%d %H:%M:%S.%f")

# Save the processed DataFrames (optional)
webcam_df.to_csv("processed_webcam.csv", index=False)
zed2_df.to_csv("processed_zed2.csv", index=False)
images_df.to_csv("processed_images.csv", index=False)
opc_df.to_csv("processed_opc.csv", index=False)

# Display processed DataFrames for verification
print("Processed Webcam DataFrame:")
print(webcam_df.head())

print("\nProcessed ZED2 DataFrame:")
print(zed2_df.head())

print("\nProcessed Combined Images DataFrame:")
print(images_df.head())

print("\nProcessed OPC DataFrame:")
print(opc_df.head())


In [ ]:
import pandas as pd
import numpy as np

def truncate_timestamps(opc_df, webcam_df):
    """
    Truncate the Timestamp column in all DataFrames to seconds.
    This ensures more overlaps between the DataFrames during synchronization.
    """
    for df in [opc_df, webcam_df]:
        df["Timestamp"] = pd.to_datetime(df["Timestamp"]).dt.floor("s")
    return opc_df, webcam_df

def aggregate_opc_data(opc_df):
    """
    Aggregate OPC DataFrame to have a single entry per Timestamp by calculating averages.
    """
    # Drop the SecondaryIndex as it's no longer needed
    opc_df = opc_df.drop(columns=["SecondaryIndex"], errors="ignore")

    # Group by Timestamp and calculate the mean for numerical columns
    aggregated_opc_df = opc_df.groupby("Timestamp").mean().reset_index()

    return aggregated_opc_df

def synchronize_opc_to_webcam(opc_df, webcam_df):
    """
    Ensure OPC DataFrame matches Webcam DataFrame timestamps.
    Fill missing OPC data with NaN for unmatched timestamps in Webcam DataFrame.
    """
    # Merge webcam timestamps with OPC data
    synchronized_df = pd.merge(
        webcam_df[["Timestamp"]],  # Use only timestamps from webcam
        opc_df,
        on="Timestamp",
        how="left"  # Keep all webcam timestamps
    )
    return synchronized_df

# Apply truncation to timestamps
opc_df, webcam_df = truncate_timestamps(opc_df, webcam_df)

# Aggregate OPC data to a single entry per timestamp
opc_df = aggregate_opc_data(opc_df)

# Synchronize OPC data to match Webcam timestamps
opc_synced = synchronize_opc_to_webcam(opc_df, webcam_df)

# Debug outputs
print(f"Aggregated OPC DataFrame length: {len(opc_df)}")
print(f"Synchronized OPC DataFrame length: {len(opc_synced)}")
print(f"Webcam DataFrame length: {len(webcam_df)}")
print(opc_synced.head())


In [ ]:

print("Synchronized OPC DataFrame:")
print(opc_synced['Timestamp'])

print("\nSynchronized Webcam DataFrame:")
print(webcam_df['Timestamp'])
"""
print("\nSynchronized ZED2 DataFrame:")
print(zed2_synced)

"""

drop flags and parameters that dont show any changes or are identical to another flag/parameter

In [ ]:
opc_alt = opc_synced.drop(columns=['Parameter1', 'Parameter2', 'Parameter4', 'Parameter5', 'Parameter9', 'Parameter11', 'Flag1', 'Flag7', 'Flag8', 'Flag9', 'Flag11'])
print(opc_alt)

In [13]:
opc_alt.to_csv(r'C:\Users\Public\Desktop\opc_alt.csv')

display remaining opc data visually

In [ ]:
import matplotlib.pyplot as plt

def visualize_opc_data(opc_df):
    """
    Visualize each column in the OPC data with individual line plots.
    
    Parameters:
    - opc_df: DataFrame containing the OPC data
    """
    # Drop the 'Timestamp' column to only plot the numerical/categorical data
    columns_to_plot = opc_df.drop(columns=['Timestamp'])
    
    # Create a figure with subplots for each column
    num_columns = len(columns_to_plot.columns)
    fig, axs = plt.subplots(num_columns, 1, figsize=(12, 4 * num_columns), sharex=True)
    
    if num_columns == 1:
        axs = [axs]  # Ensure axs is iterable if there's only one column

    # Iterate through the columns and plot each
    for i, column in enumerate(columns_to_plot.columns):
        axs[i].plot(opc_df['Timestamp'], columns_to_plot[column], label=column, linewidth=0.8)
        axs[i].set_title(f"{column} Over Time", fontsize=12)
        axs[i].set_ylabel(column)
        axs[i].grid(True)
        axs[i].legend(loc='upper right')

    # Set shared X-axis label
    axs[-1].set_xlabel("Timestamp")
    
    # Adjust layout for better spacing
    plt.tight_layout()
    plt.show()

# Example Usage:
visualize_opc_data(opc_alt)


# CNN

prepare opc data for CNN

In [10]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [11]:
opc_cleaned = opc_alt
scaler = MinMaxScaler()
opc_cleaned.iloc[:, 1:] = scaler.fit_transform(opc_cleaned.iloc[:, 1:])
#print(opc_cleaned)
#visualize_opc_data(opc_cleaned)

In [ ]:

opc_cleaned['Timestamp'] = pd.to_datetime(opc_cleaned['Timestamp'])
opc_cleaned['Seconds'] = opc_cleaned['Timestamp'].dt.hour * 3600 + \
                         opc_cleaned['Timestamp'].dt.minute * 60 + \
                         opc_cleaned['Timestamp'].dt.second
opc_cleaned.drop(columns=["Timestamp"])


In [ ]:
print(opc_cleaned)

In [14]:
window_size = 5
features = opc_cleaned.iloc[:, 1:].values  # Exclude the seconds column
sequences = []
for i in range(len(features) - window_size):
    sequences.append(features[i:i+window_size])

opc_sequences = np.array(sequences)  # Shape: (samples, timesteps, features)


In [15]:
np.save('opc_sequences.npy', opc_sequences)

load images

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

# Paths
output_folder = r"D:\PAIND\DATA\Processed"
os.makedirs(output_folder, exist_ok=True)
image_folder = r"D:\PAIND\DATA\20240527_Data_Prozess_01\webcam\labeled_regions\inside_downscaled"

# Fixed image size for CNN
IMAGE_SIZE = (224, 126)

# Load the synchronized webcam DataFrame
#webcam_df = webcam_synced

# Parameters for batch processing
batch_size = 1000  # Number of images to process in one batch
num_batches = (len(webcam_df) // batch_size) + 1

# Process images in batches
for batch_idx in range(num_batches):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, len(webcam_df))
    batch_df = webcam_df.iloc[start_idx:end_idx]

    processed_images = []  # Placeholder for the current batch

    for _, row in tqdm(batch_df.iterrows(), total=len(batch_df), desc=f"Processing Batch {batch_idx + 1}/{num_batches}"):
        image_path = os.path.join(image_folder, row["image_name"])
        
        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Image not found: {image_path}. Skipping...")
            continue
        
        # Resize and normalize the image
        image_resized = cv2.resize(image, IMAGE_SIZE)
        image_normalized = image_resized / 255.0  # Normalize to [0, 1]
        
        processed_images.append(image_normalized)

    # Convert list to NumPy array and save the batch
    batch_array = np.array(processed_images)
    batch_output_path = os.path.join(output_folder, f"processed_images_batch_{batch_idx + 1}.npy")
    np.save(batch_output_path, batch_array)

    print(f"Processed batch {batch_idx + 1}/{num_batches} and saved to {batch_output_path}.")

print("All batches processed and saved.")


merge opc data & images

In [ ]:
import numpy as np

# Path to one of the processed image batches
npy_file_path = r"D:\PAIND\DATA\Processed\processed_images_batch_2.npy"

# Load the .npy file
try:
    loaded_data = np.load(npy_file_path)
    # Check the shape and content of the loaded data
    print(f"Loaded data shape: {loaded_data.shape}")
    print(f"Sample data (first entry):\n{loaded_data[0]}")
except Exception as e:
    print(f"Error loading .npy file: {e}")


In [ ]:
import os
import numpy as np
import pandas as pd

# Paths
processed_images_folder = r"D:\PAIND\DATA\Processed"  # Folder containing processed image batches
output_folder = r"D:\PAIND\DATA\Processed"

# Load synchronized OPC DataFrame
opc_df = opc_cleaned  # Ensure this is synchronized and sorted

# Placeholder for merged results
merged_batches = []

# List batch files
batch_files = sorted([file for file in os.listdir(processed_images_folder) if file.startswith("processed_images_batch_") and file.endswith(".npy")])

# Check batch size from the first `.npy` file
batch_size = np.load(os.path.join(processed_images_folder, batch_files[0])).shape[0]

# Process batches based on order
for batch_idx, batch_file in enumerate(batch_files):
    # Load the processed image batch
    batch_path = os.path.join(processed_images_folder, batch_file)
    image_batch = np.load(batch_path)

    # Determine the slice of OPC DataFrame corresponding to this batch
    start_idx = batch_idx * batch_size
    end_idx = start_idx + len(image_batch)  # Account for last batch which may be smaller
    opc_batch = opc_df.iloc[start_idx:end_idx].reset_index(drop=True)

    # Verify alignment
    try:
        assert len(opc_batch) == len(image_batch), f"Mismatch in batch {batch_idx + 1}!"
        # Append the merged batch to results
        merged_batches.append((opc_batch, image_batch))
    except:
        print('exception at {batch_idx}')
        break

# Save merged data and images
for batch_idx, (opc_batch, image_batch) in enumerate(merged_batches):
    try:
        # Save the merged OPC batch as a CSV
        merged_csv_path = os.path.join(output_folder, f"merged_opc_data_batch_{batch_idx + 1}.csv")
        opc_batch.to_csv(merged_csv_path, index=False)
        
        # Save the corresponding image batch
        merged_image_path = os.path.join(output_folder, f"merged_images_batch_{batch_idx + 1}.npy")
        np.save(merged_image_path, image_batch)
    except:
        print('exception at {batch_idx}')
        break

print("All batches merged and saved.")


In [ ]:
# Define split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Ensure the ratios sum to 1
assert train_ratio + val_ratio + test_ratio == 1, "Ratios must sum to 1!"

# Determine split indices
num_batches = len(merged_batches)
train_end = int(train_ratio * num_batches)
val_end = int((train_ratio + val_ratio) * num_batches)

# Split the merged_batches
train_batches = merged_batches[:train_end]
val_batches = merged_batches[train_end:val_end]
test_batches = merged_batches[val_end:]

# Save the splits
output_folder = r"D:\PAIND\DATA\Processed"

def save_split_batches(split_batches, split_name):
    os.makedirs(os.path.join(output_folder, split_name), exist_ok=True)
    for idx, (merged_batch, image_batch) in enumerate(split_batches):
        # Save the merged batch as a CSV
        merged_csv_path = os.path.join(output_folder, split_name, f"{split_name}_data_batch_{idx + 1}.csv")
        merged_batch.to_csv(merged_csv_path, index=False)

        # Save the corresponding image batch
        merged_image_path = os.path.join(output_folder, split_name, f"{split_name}_images_batch_{idx + 1}.npy")
        np.save(merged_image_path, image_batch)

# Save each split
save_split_batches(train_batches, "train")
save_split_batches(val_batches, "val")
save_split_batches(test_batches, "test")

# Print summary
print(f"Training Batches: {len(train_batches)}")
print(f"Validation Batches: {len(val_batches)}")
print(f"Test Batches: {len(test_batches)}")


In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
def create_cnn(input_shape, output_size):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(output_size, activation='linear')  
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Initialize model
input_shape = (126, 224, 3)  
output_size = 12  #
cnn_model = create_cnn(input_shape, output_size)

# Training function
def train_cnn_on_batches(model, train_folder, val_folder, epochs, batch_size):
    """
    Train the CNN model batch-by-batch across epochs.
    """
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        train_batches = sorted([f for f in os.listdir(train_folder) if f.startswith("train_data_batch")])

        for batch_file in train_batches:
            # Load batch data
            batch_idx = int(batch_file.split("_")[-1].split(".")[0])
            train_data = pd.read_csv(os.path.join(train_folder, batch_file))
            train_images = np.load(os.path.join(train_folder, f"train_images_batch_{batch_idx}.npy"))

            # Prepare training inputs and outputs
            x_train = train_images  # Image batch
            y_train = train_data.iloc[:, 1:].values  # OPC features (drop timestamp)

            # Train the model on this batch
            model.fit(x_train, y_train, batch_size=batch_size, verbose=1)

        # Validate after every epoch
        validate_cnn_on_batches(model, val_folder)

# Validation function
def validate_cnn_on_batches(model, val_folder):
    """
    Validate the CNN model on all validation batches.
    """
    val_batches = sorted([f for f in os.listdir(val_folder) if f.startswith("val_data_batch")])
    total_loss, total_mae, total_samples = 0, 0, 0

    for batch_file in val_batches:
        # Load validation batch
        batch_idx = int(batch_file.split("_")[-1].split(".")[0])
        val_data = pd.read_csv(os.path.join(val_folder, batch_file))
        val_images = np.load(os.path.join(val_folder, f"val_images_batch_{batch_idx}.npy"))

        # Prepare validation inputs and outputs
        x_val = val_images
        y_val = val_data.iloc[:, 1:].values

        # Evaluate this batch
        loss, mae = model.evaluate(x_val, y_val, verbose=1)
        batch_size = len(val_data)
        total_loss += loss * batch_size
        total_mae += mae * batch_size
        total_samples += batch_size

    # Compute final metrics
    final_loss = total_loss / total_samples
    final_mae = total_mae / total_samples
    print(f"Validation Loss: {final_loss:.4f}, Validation MAE: {final_mae:.4f}")

# Paths
train_folder = r"D:\PAIND\DATA\Processed\train"
val_folder = r"D:\PAIND\DATA\Processed\val"
test_folder = r"D:\PAIND\DATA\Processed\test"

# Train the model
train_cnn_on_batches(cnn_model, train_folder, val_folder, epochs=50, batch_size=32)

# Save the trained model
model_path = r"D:\PAIND\DATA\Processed\cnn.keras"
cnn_model.save(model_path)
print(f"Model saved to {model_path}")


avg. max RAM used for whole script: 54GB +/- 1GB

In [ ]:
from keras.models import load_model

# Load the trained model
trained_model = load_model(model_path)

# Get test batch files
batch_files = sorted([file for file in os.listdir(test_folder) if file.endswith(".csv")])

# Iterate over test batches
for batch_file in batch_files:
    # Load the test data CSV and corresponding image .npy
    batch_csv_path = os.path.join(test_folder, batch_file)
    batch_data = pd.read_csv(batch_csv_path)

    # Replace "data" with "images" to find the correct .npy file for images
    image_batch_path = batch_csv_path.replace("_data_", "_images_").replace(".csv", ".npy")
    x_test = np.load(image_batch_path)  # Load the image batch

    # OPC features
    y_test = batch_data.iloc[:, 1:].values  # Drop the Timestamp column

    # Evaluate on this batch
    results = trained_model.evaluate(x_test, y_test, verbose=1)
    print(f"Batch {batch_file}: Loss = {results[0]}, MAE = {results[1]}")


In [ ]:
# Predict on test batch
predictions = trained_model.predict(x_test)

# Save predictions alongside original OPC data
predictions_df = pd.DataFrame(predictions, columns=[f"Predicted_{col}" for col in batch_data.columns[1:]])
results_df = pd.concat([batch_data, predictions_df], axis=1)

# Save to a file
output_predictions_path = r"D:\PAIND\DATA\Processed\predictions.csv"
results_df.to_csv(output_predictions_path, index=False)
print(f"Predictions saved to: {output_predictions_path}")


In [ ]:
visualize_opc_data(results_df)

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd

# Paths
test_folder = r"D:\PAIND\DATA\Processed\test"

# Load the trained model
from keras.models import load_model
model_path = r"D:\PAIND\DATA\Processed\cnn.keras"
trained_model = load_model(model_path)

# Initialize lists to store results
losses = []
maes = []
batch_indices = []

# Get test batch files
batch_files = sorted([file for file in os.listdir(test_folder) if file.endswith(".csv")])

# Iterate over test batches and collect loss and MAE
for batch_file in batch_files:
    # Load the test data CSV and corresponding image .npy
    batch_csv_path = os.path.join(test_folder, batch_file)
    batch_data = pd.read_csv(batch_csv_path)
    image_batch_path = batch_csv_path.replace("_data_", "_images_").replace(".csv", ".npy")
    x_test = np.load(image_batch_path)  # Load the image batch
    y_test = batch_data.iloc[:, 1:].values  # Drop the Timestamp column

    # Evaluate on this batch
    loss, mae = trained_model.evaluate(x_test, y_test, verbose=0)  # Suppress verbose output
    losses.append(loss)
    maes.append(mae)
    batch_indices.append(batch_file.split("_")[-1].replace(".csv", ""))  # Extract batch index

# Convert batch indices to integers for sorting
batch_indices = [int(idx) for idx in batch_indices]

# Plot Loss and MAE
plt.figure(figsize=(10, 6))

# Plot Loss
plt.subplot(2, 1, 1)
plt.plot(batch_indices, losses, marker="o", label="Loss")
plt.title("Loss and MAE Across Test Batches")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()

# Plot MAE
plt.subplot(2, 1, 2)
plt.plot(batch_indices, maes, marker="o", label="MAE", color="orange")
plt.xlabel("Batch Index")
plt.ylabel("Mean Absolute Error (MAE)")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import re

# File path to your loss and MAE data
file_path = r"D:\PAIND\src\loss_mae.txt"

def parse_loss_mae_file(file_path):
    """
    Parse the loss and MAE file to extract values for each epoch.
    """
    # Regular expression to capture loss and MAE values
    batch_pattern = re.compile(r"loss: ([\d.]+) - mae: ([\d.]+)")
    validation_pattern = re.compile(r"Validation Loss: ([\d.]+), Validation MAE: ([\d.]+)")
    
    epoch_losses, epoch_maes, val_losses, val_maes = [], [], [], []

    # Open the file with error handling for decoding
    with open(file_path, "r", encoding="utf-8", errors="ignore") as file:
        for line in file:
            # Match batch-level metrics
            batch_match = batch_pattern.search(line)
            if batch_match:
                loss, mae = map(float, batch_match.groups())
                epoch_losses.append(loss)
                epoch_maes.append(mae)

            # Match validation metrics
            val_match = validation_pattern.search(line)
            if val_match:
                val_loss, val_mae = map(float, val_match.groups())
                val_losses.append(val_loss)
                val_maes.append(val_mae)

    return epoch_losses, epoch_maes, val_losses, val_maes

# Parse the file
epoch_losses, epoch_maes, val_losses, val_maes = parse_loss_mae_file(file_path)

# Plot the metrics
epochs = range(1, len(val_losses) + 1)

# Plot Loss and MAE
plt.figure(figsize=(10, 6))

# Plot Loss
plt.subplot(2, 1, 1)
plt.plot(epochs, val_losses, marker="o", label="Loss")
plt.title("Loss and MAE Across Test Batches")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()

# Plot MAE
plt.subplot(2, 1, 2)
plt.plot(epochs, val_maes, marker="o", label="MAE", color="orange")
plt.xlabel("Batch Index")
plt.ylabel("Mean Absolute Error (MAE)")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()